In [4]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
%matplotlib ipympl

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.6f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})

In [5]:
GHz = 1e9
fF = 1e-15
nH = 1e-9

In [6]:
def coupled_fluxoniums(L1, C1, EJ1, L2, C2, EJ2, CC):
    loop1 = sq.Loop(0.5)
    loop2 = sq.Loop(0.5)
    
    C_01 = sq.Capacitor(C1, 'fF')
    L_01 = sq.Inductor (L1, 'nH', loops=[loop1])
    EJ_01  = sq.Junction(EJ1, 'GHz', loops=[loop1])
    
    C_02 = sq.Capacitor(C2, 'fF')
    L_02 = sq.Inductor (L2, 'nH', loops=[loop2])
    EJ_02  = sq.Junction(EJ2, 'GHz', loops=[loop2])
    
    C_12 = sq.Capacitor(CC, 'fF')
    
    elements = {(0, 1): [L_01, C_01, EJ_01],
                (0, 2): [L_02, C_02, EJ_02],
                (1, 2): [C_12], }
    return sq.Circuit(elements)

def fluxonium(L, C, EJ):
    loop = sq.Loop(0.5)
    C_01   = sq.Capacitor(C, 'fF')
    L_01   = sq.Inductor (L, 'nH', loops=[loop])
    EJ_01  = sq.Junction(EJ, 'GHz', loops=[loop])
    elements = {(0, 1): [L_01, C_01, EJ_01]}

    return sq.Circuit(elements)


In [7]:
n_eig = 5
trunc_num = 25

In [8]:
L1  = 3
C1  = 3
EJ1 = 5
L2  = 1
C2  = 5
EJ2 = 5

CC = 2

In [12]:
coupled_res_full = coupled_fluxoniums(L1=L1, C1=C1, EJ1=EJ1, L2=L2, C2=C2, EJ2=EJ2, CC=CC)
coupled_res_full .set_trunc_nums([trunc_num, trunc_num])
H_coupled_res_full = coupled_res_full.hamiltonian()

In [13]:
C_mat = np.array([[C1 + CC, -CC],
                  [-CC, C2 + CC]])
C_inv = np.linalg.inv(C_mat)
C1_tilde = C_inv[0,0]**-1
C2_tilde = C_inv[1,1]**-1

In [15]:
fluxonium_1 = fluxonium(L=L1, C=C1_tilde, EJ=EJ1)
fluxonium_2 = fluxonium(L=L2, C=C2_tilde, EJ=EJ2)
fluxonium_1.set_trunc_nums([trunc_num])
fluxonium_2.set_trunc_nums([trunc_num])

In [17]:
Q_1 = fluxonium_1.charge_op(0)
Q_2 = fluxonium_2.charge_op(0)
Φ_1 = fluxonium_1.flux_op(0)
Φ_2 = fluxonium_2.flux_op(0)

H_1 = fluxonium_1.hamiltonian()
# H_1 = 1/2 * C1_tilde**-1 * fF**-1 * Q_1**2 + 1/2 * L1**-1 * nH**-1 * Φ_1**2 
H_2 = fluxonium_2.hamiltonian()
# H_2 = 1/2 * C2_tilde**-1 * fF**-1 * Q_2**2 + 1/2 * L2**-1 * nH**-1 * Φ_2**2 

I = qt.identity(H_1.shape[0])
H_coupled_res_comp = qt.tensor(H_1,I) + qt.tensor(I,H_2) + C_inv[0,1]* fF**-1 * qt.tensor(Q_1,Q_2)

In [18]:
E_coupled_res_comp = sq_ext.diag(H_coupled_res_comp, n_eig, remove_ground=True)[0]
E_coupled_res_full = sq_ext.diag(H_coupled_res_full, n_eig, remove_ground=True)[0]

In [19]:
E_coupled_res_comp, E_coupled_res_full

(array([0.000000, 38.035963, 65.492617, 76.297202, 103.637949]),
 array([0.000000, 38.035963, 65.492617, 76.297202, 103.637949]))